# QuantX SDK はじめの一歩

Special thanks, shinseitaroさん!

このドキュメントは shinseitaroさんに作成いただいたものをSmartTradeにて編集させていただきました。ありがとうございます！

このインストラクションは未完成です。メモ程度なので、わかりにくいと思います。ご意見いただければ書き直しますので教えてくださいm(_ _)m

---

### アクセスキーを取得

sdk を使うためには，アクセスキーとシークレットキーが必要です．

アクセスキーとシークレットキーは、quantxにログインして，https://factory.quantx.io/settings/token から取得してください．

取得したアクセスキーとシークレットキーを変数に格納しておきましょう．


In [5]:
# public_keyとsecret_keyは各自ご自身のキーで置き換えてください
access_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
secret_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"


---
### 新規プロジェクトを作成


In [6]:
# sdk をインポート
from quantx_sdk import QX

# 新規プロジェクト作成
qx = QX(access_key, secret_key)

# 新規プロジェクト名。５文字以上の適当な文字列
new_name = "an new project"

# 新規プロジェクトを作成。
prj = qx.new_project(new_name)



上記を実行した後,https://factory.quantx.io/projects にログインしてみてください．

`new_name` の名前を持つ新しいプロジェクトが作成されているハズです．

そのプロジェクトを開くと，サンプルコードも挿入されています.

**つまり， `qx.new_project(new_name)` の実行　=　ウェブで`[新規プロジェクトを作成]`ボタンを押す事**

です．

ウェブ上のIDEを使って入出力を行う代わりに，jupyter lab を使っているだけです．（追加機能も少し有り）

ですので，jupyter lab から実行するたびに，Webの方も更新されています．

---

### ハッシュ番号（プロジェクトID）

では，先ほどの新規プロジェクト作成セルを2−3回実行してみましょう．

その後，https://factory.quantx.io/projects を確認してみてください．

同じ名前のプロジェクトが複数作られていると思います．

QuantXではプロジェクトの管理をハッシュ番号で行っています．よって，プロジェクト名はユニークである必要はありません．プロジェクトの新規作成が行われる度にユニークなハッシュ番号が割り振られます．

つまりひとつのプロジェクトを使いまわしたい場合は，このハッシュ番号を使ってプロジェクトを指定することになります．

ハッシュ番号はプロジェクトのURLの _developer_ 以下の文字列と同じです．例： https://factory.quantx.io/developer/xxxxxxxxxxxxxxxxxxxxxxxxxxxx



In [12]:
# ハッシュ番号を格納，確認．
prjct_hash = prj.project_hash
prjct_hash


'd7058b6e0eb741a0bfca5a2b29c0f20a'

**確認！**　取得出来たハッシュタグは，ウェブ上のプロジェクトURLの _developer_ 以下の文字列と一致します．確認してください．

（新規作成を複数回した場合は，最も新しいプロジェクトのハッシュ番号と一致します．他のプロジェクトは要らないので，ウェブ上で削除してください）

---
### コード取得

新規作成したプロジェクトにはサンプルコードが自動挿入されています．そのサンプルコードを出力してみましょう．

コードは文字列で出力されますが，キレイなフォーマットでは出力されません．



In [10]:
prj.source()

'############################################################################\n# Sample Algorithm\n\ndef initialize(ctx):\n    # 設定\n    ctx.logger.debug("initialize() called")\n    ctx.configure(\n      channels={          # 利用チャンネル\n        "jp.stock": {\n          "symbols": [\n            "jp.stock.1305",\n            "jp.stock.9984",\n            "jp.stock.9983",\n            "jp.stock.7201",\n            "jp.stock.9201",\n            "jp.stock.9202",\n            "jp.stock.7203"\n          ],\n          "columns": [\n            #"open_price_adj",    # 始値(株式分割調整後)\n            #"high_price_adj",    # 高値(株式分割調整後)\n            #"low_price_adj",     # 安値(株式分割調整後)\n            #"volume_adj",         # 出来高\n            #"txn_volume",         # 売買代金\n            "close_price",        # 終値\n            "close_price_adj",    # 終値(株式分割調整後) \n          ]\n        }\n      }\n    )\n\n    def _my_signal(data):\n        return {\n        }\n\n    # シグナル登録\n    ctx.regist_signal("my_signal", 

---

### プロジェクトパラメータ取得

テスト期間，当初資金額を出力してみましょう．ウェブでの右上に出ている「期間」「初期資金量」「選択中のエンジン」と同じ情報が確認出来ます．

In [11]:
prj.current_backtest_params()

{'engine': 'maron-0.0.1b',
 'to_date': '2017-03-31',
 'from_date': '2015-01-01',
 'capital_base': 10000000}

---

### バックテスト実行

サンプルコードをバックテストしてみましょう．

（ただし，新規作成した際に自動で挿入されているサンプルコードでは，売買取引の部分がコメントアウトされていますので，結果が出力されません．）

まずは，`backtest()` でバックテストを実行し，バックテストオブジェクトを取得します．

そのオブジェクトにたいして `join()` を実行して結果を得ます．

`summary()` で結果のサマリーを取得します．



In [15]:
# backtest の実行. backtest object を取得
bt = prj.backtest()

# backtest result インスタンスを作成
join = bt.join()

# 結果のサマリーを取得
summary = join.summary()

# summary を見ると何も結果無し。
summary

{'2015': 0,
 '2016': 0,
 '2017': 0,
 'Volatility': 0,
 'SharpeRatio': None,
 'MaxDrawdown': 0,
 'Alpha': 0,
 'Beta': [0],
 'TradingDays': 550,
 'ReturnValue': 10000000,
 'ReturnRatio': '0.0',
 'SignalCount': 0}

---

## 自分のアルゴリズムをUploadして実行

では，簡単なアルゴリズムをUploadして結果を出力させてみましょう．今回は[RSIのアルゴリズム](https://qiita.com/shinseitaro/items/6a699cf795c7254b0c84)を使ってみます．

アルゴリズムは文字列で渡します．Python では，**ダブルクォーテーション3つでくくる**と複数行に渡る文字列を記述する事が出来ます．






In [20]:
#簡単なバックテストコード
source = """

import numpy as np 
import talib as ta 


def initialize(ctx):
    # 設定
    ctx.target = 0.10
    ctx.period = 5
    ctx.codes = [1605, 1925, 2503, 4519, 4911, 6301, 6752, 7741, 8001] 
    ctx.symbol_list  = ["jp.stock.{}".format(code) for code in ctx.codes]


    ctx.configure(
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols":ctx.symbol_list,
          "columns": ["close_price_adj",    # 終値(株式分割調整後) 
          ]}})

    def _RSI(data):

      df_close = data["close_price_adj"].fillna(method='ffill')
      # memo 参照
      df_rsi = df_close.apply(lambda s: ta.RSI(s.values.astype(np.double), timeperiod=ctx.period),axis=0)

      buy_sig = df_rsi < 30.0 
      sell_sig = df_rsi > 70.0 

      return {
        "rsi": df_rsi,
        "buy:sig": buy_sig, 
        "sell:sig": sell_sig, 

        }

    # シグナル登録
    ctx.regist_signal("RSI", _RSI)

def handle_signals(ctx, date, current):
    df = current.copy()

    # 買いシグナル
    df_long = df[df["buy:sig"]]
    if not df_long.empty:
      for (sym, val) in df_long.iterrows(): 
        sec = ctx.getSecurity(sym)
        msg = "買いシグナル"
        sec.order_target_percent(ctx.target, comment= msg)

    # 売り（ポジションクローズ）シグナル
    df_sell = df[df["sell:sig"]]
    if not df_sell.empty:
      for (sym, val) in df_sell.iterrows(): 
        sec = ctx.getSecurity(sym)
        msg = "売りシグナル"
        sec.order_target_percent(0, comment= msg)
        

"""

次にプロジェクトを指定して，コードをUploadします．

先ほど作った `prjct_hash` を利用して，プロジェクトオブジェクトを作ります．

そのオブジェクトに対して， `upload_source()` メソッドでコードをUploadします．

`'code': 200` が返ってくればOKです．



In [21]:
# hash番号を与えてプロジェクトオブジェクトを作ります
prj = qx.project(prjct_hash)

# プロジェクトにコードをUpload
prj.upload_source(source)


{'code': 200,
 'result': {'hash': 'd7058b6e0eb741a0bfca5a2b29c0f20a',
  'updated_at': '2018-09-20T06:43:46.821Z'}}

ここで，ウェブのプロジェクトを確認してみましょう．コードが書き換えられている事が確認出来たと思います．

次にバックテストを実行します．上記の通り `prj.backtest()` すれば良いのですが，プロジェクトパラメータを変更してみましょう．


In [23]:
# バックテストパラメータ
bt_parameter = {
    'engine': 'maron-0.0.1b',
    'from_date': '2017-01-01',
    'to_date': '2018-09-01',
    'capital_base': 10000000}

# backtest の実行. 
bt = prj.backtest(bt_parameter)

## backtest result インスタンスを作成
join = bt.join()

## 結果のサマリーを取得
summary = join.summary()
summary

銘柄毎のsummaryを出すには `symbol_summary()` を使います

In [25]:
join.symbol_summary()

,symbol,name,sector,txn_count,position_count,valuation,return
0,jp.stock.1605,国際石油開発帝石(株),鉱業,24,0,0.0,-7250.0
1,jp.stock.1925,大和ハウス工業(株),建設業,18,300,1016700.0,9600.0
2,jp.stock.2503,キリンホールディングス(株),食料品,24,0,0.0,229300.0
3,jp.stock.4519,中外製薬(株),医薬品,21,0,0.0,320500.0
4,jp.stock.4911,(株)資生堂,化学,22,0,0.0,411200.0
5,jp.stock.6301,コマツ,機械,20,0,0.0,75550.0
6,jp.stock.6752,パナソニック(株),電気機器,23,800,1061200.0,38300.0
7,jp.stock.7741,ＨＯＹＡ(株),精密機器,21,100,649700.0,220200.0
8,jp.stock.8001,伊藤忠商事(株),卸売業,18,0,0.0,87500.0


### 動的にコードを変更（アルゴリズムをテンプレート化）

`source` をテンプレート化して，銘柄を動的に変えて評価していきましょう。

`ctx.codes = [1605, 1925, 2503, 4519, 4911, 6301, 6752, 7741, 8001] ` で指定している銘柄を動的に変更していきます．

テンプレートで動的に変更したい箇所を `{{ 引数名 }}` と記述し，レンダーして使います．

まずは `my_template` を作ります．



In [27]:
my_template = """

import numpy as np 
import talib as ta 


def initialize(ctx):
    # 設定
    ctx.target = 0.10
    ctx.period = 5
    ctx.codes = {{symbols}} # <= 動的に銘柄を変更する
    ctx.symbol_list  = ["jp.stock.{}".format(code) for code in ctx.codes]


    ctx.configure(
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols":ctx.symbol_list,
          "columns": ["close_price_adj",    # 終値(株式分割調整後) 
          ]}})

    def _RSI(data):

      df_close = data["close_price_adj"].fillna(method='ffill')
      # memo 参照
      df_rsi = df_close.apply(lambda s: ta.RSI(s.values.astype(np.double), timeperiod=ctx.period),axis=0)

      buy_sig = df_rsi < 30.0 
      sell_sig = df_rsi > 70.0 

      return {
        "rsi": df_rsi,
        "buy:sig": buy_sig, 
        "sell:sig": sell_sig, 

        }

    # シグナル登録
    ctx.regist_signal("RSI", _RSI)

def handle_signals(ctx, date, current):
    df = current.copy()

    # 買いシグナル
    df_long = df[df["buy:sig"]]
    if not df_long.empty:
      for (sym, val) in df_long.iterrows(): 
        sec = ctx.getSecurity(sym)
        msg = "買いシグナル"
        sec.order_target_percent(ctx.target, comment= msg)

    # 売り（ポジションクローズ）シグナル
    df_sell = df[df["sell:sig"]]
    if not df_sell.empty:
      for (sym, val) in df_sell.iterrows(): 
        sec = ctx.getSecurity(sym)
        msg = "売りシグナル"
        sec.order_target_percent(0, comment= msg)
        

"""

次に銘柄リストを作り，テンプレートにレンダーします．

そのコードを実行してみましょう．



In [30]:
from jinja2 import Template

symbols = [3276, 2117, 4581, 6620, 7022]
source2 = Template(my_template).render({'symbols': symbols})

# バックテストパラメータ
bt_parameter = {
    'engine': 'maron-0.0.1b',
    'from_date': '2017-01-01',
    'to_date': '2018-09-01',
    'capital_base': 10000000}

prj = qx.project(prjct_hash)
prj.upload_source(source2)
bt = prj.backtest()
join = bt.join()
summary = join.summary()
symbol_summary = join.symbol_summary()


In [32]:
## 結果を出してみましょう。
summary

{'2015': 0.06088000000000093,
 '2016': 0.08384548397953107,
 '2017': 0.022298948255412343,
 'Volatility': 0.09486735492231595,
 'SharpeRatio': 0.8295474803974042,
 'MaxDrawdown': -0.11324869467645127,
 'Alpha': 0.06047005869060031,
 'Beta': [0.2782649306611768],
 'TradingDays': 550,
 'ReturnValue': 11754699.96650815,
 'ReturnRatio': '17.5',
 'SignalCount': 215}

In [33]:
## 銘柄ごとのサマリーを見ると指定した銘柄が実行されているのがわかりますね．
join.symbol_summary()

,symbol,name,sector,txn_count,position_count,valuation,return
0,jp.stock.2117,日新製糖(株),食料品,18,599.999982,1.123800e+06,410599.966508
1,jp.stock.3276,日本管理センター(株),不動産業,36,0.000000,0.000000e+00,-78300.000000
2,jp.stock.4581,大正製薬ホールディングス(株),医薬品,28,0.000000,0.000000e+00,300000.000000
3,jp.stock.6620,宮越ホールディングス(株),電気機器,51,0.000000,0.000000e+00,600500.000000
4,jp.stock.7022,サノヤスホールディングス(株),輸送用機器,82,4000.000000,1.160000e+06,521900.000000


ウェブ上でもコードが書き換わっていることを確認してみてください．
